##Challenge 3: Alcohol Consumption

Loading Essential Libraries For The Analysis

In [2]:
import pandas as pd
from siuba import *
import countrycode

Reading Dataset From Remote Server

In [3]:
wine =pd.read_csv('https://raw.githubusercontent.com/justmarkham/DAT8/master/data/drinks.csv')

Let us see the whole picture of the data

In [4]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   continent                     170 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


Way the data has been represented to us we can manipulate the data in such way so we don't have to worry about missing value. To have a eagle eye vision of the data we will check the head of the data now.

In [5]:
wine.head(10)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF
5,Antigua & Barbuda,102,128,45,4.9,NaN
6,Argentina,193,25,221,8.3,SA
7,Armenia,21,179,11,3.8,EU
8,Australia,261,72,212,10.4,OC
9,Austria,279,75,191,9.7,EU


We can clearly see that the continent column is missing some(totally 23) values. It is easily understandable that the 'continent' column is depended on the 'Country' column. We will drop the 'continent' column and make our own new 'Continent' column based on the 'Country' column

In [6]:
wine = wine >> select(~_.continent)

In [7]:
wine.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol
0,Afghanistan,0,0,0,0.0
1,Albania,89,132,54,4.9
2,Algeria,25,0,14,0.7
3,Andorra,245,138,312,12.4
4,Angola,217,57,45,5.9


Adding New Column With Automated Continent Name

In [9]:
wine['Continent'] = wine['country'].apply(countrycode.countrycode, origin='country.name', destination='continent')

In [10]:
wine.head(10)

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,Continent
0,Afghanistan,0,0,0,0.0,Asia
1,Albania,89,132,54,4.9,Europe
2,Algeria,25,0,14,0.7,Africa
3,Andorra,245,138,312,12.4,Europe
4,Angola,217,57,45,5.9,Africa
5,Antigua & Barbuda,102,128,45,4.9,Americas
6,Argentina,193,25,221,8.3,Americas
7,Armenia,21,179,11,3.8,Asia
8,Australia,261,72,212,10.4,Oceania
9,Austria,279,75,191,9.7,Europe


In [11]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   Continent                     192 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


With some internet research we found that Micronesia includes 2100 island that belongs to Oceania is not actually considered as a country. Well we need to replace that NA value with "Oceania" string.

In [14]:
wine['Continent'] = wine['Continent'].fillna("Oceania")

In [15]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 6 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       193 non-null    object 
 1   beer_servings                 193 non-null    int64  
 2   spirit_servings               193 non-null    int64  
 3   wine_servings                 193 non-null    int64  
 4   total_litres_of_pure_alcohol  193 non-null    float64
 5   Continent                     193 non-null    object 
dtypes: float64(1), int64(3), object(2)
memory usage: 9.2+ KB


Our Data is clean and ready for the analysis



Which continent drinks more beer on average?

In [16]:
wine >> group_by(_.Continent) >> summarize(avg_beer_con = _.beer_servings.mean()) >> arrange(-_.avg_beer_con) >> head(1)

,Continent,avg_beer_con
3,Europe,206.690476


For each continent print the statistics for wine consumption.

In [17]:
wine >> group_by(_.Continent) >> summarize(
        mean_wine_servings=_.wine_servings.mean(),
        median_wine_servings=_.wine_servings.median(),
        min_wine_servings=_.wine_servings.min(),
        max_wine_servings=_.wine_servings.max(),
        std_wine_servings=_.wine_servings.std()
)

,Continent,mean_wine_servings,median_wine_servings,min_wine_servings,max_wine_servings,std_wine_servings
0,Africa,16.264151,2.0,0,233,38.846419
1,Americas,37.514286,11.0,1,221,58.231471
2,Asia,12.851064,1.0,0,149,31.564274
3,Europe,147.500000,128.5,0,370,96.812177
4,Oceania,35.625000,8.5,0,212,64.555790


Print the mean alcohol consumption per continent for every column.

In [18]:
wine >> group_by(_.Continent) >> summarize(
        mean_beer_servings=_.beer_servings.mean(),
        mean_wine_servings=_.wine_servings.mean(),
        mean_spirit_servings=_.spirit_servings.mean(),
        mean_total_litres_of_pure_alcohol=_.total_litres_of_pure_alcohol.mean()
)

,Continent,mean_beer_servings,mean_wine_servings,mean_spirit_servings,mean_total_litres_of_pure_alcohol
0,Africa,61.471698,16.264151,16.339623,3.007547
1,Americas,155.600000,37.514286,148.257143,6.102857
2,Asia,35.510638,12.851064,60.212766,2.185106
3,Europe,206.690476,147.500000,138.380952,9.061905
4,Oceania,89.687500,35.625000,58.437500,3.381250


Print the median alcohol consumption per continent for every column.

In [19]:
wine >> group_by(_.Continent) >> summarize(
        median_beer_servings=_.beer_servings.median(),
        median_wine_servings=_.wine_servings.median(),
        median_spirit_servings=_.spirit_servings.median(),
        median_total_litres_of_pure_alcohol=_.total_litres_of_pure_alcohol.median()
)

,Continent,median_beer_servings,median_wine_servings,median_spirit_servings,median_total_litres_of_pure_alcohol
0,Africa,32.0,2.0,3.0,2.30
1,Americas,159.0,11.0,124.0,6.30
2,Asia,20.0,1.0,21.0,1.40
3,Europe,227.0,128.5,124.0,10.20
4,Oceania,52.5,8.5,37.0,1.75


Analysis Complete!